# Food Security

There are over 200 definitions of food security however the most internationally accepted definition is; food security exists when all people at all-time have both physical and economic access to sufficient, safe and nutritious food that meets their dietary need and food preferences for an active and healthy life.Food security is a multidimensional concept that involves various factors that cut across social, political, environmental and economic issues. The four pillars of food security are:
1. Availability.
2. Access.
3. Utilization.
4. Stability.

The lack of a commonly accepted and comprehensive measure for food security on an international scale has been identified as one of the roadblocks on the way to the eradication of hunger and malnutrion. This research identifies a suit of indicators needed to cover the diferent dimensions of food security and uses a machine learning algorithm to generate a cluster of geographical zones of the 47 Counties in Kenya with similar food security scenarious.

## Explolatory Data Analysis

In [3]:
# Import relevant libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import sklearn as sk
from math import sqrt
import fiona
import os

In [4]:
# import data
file_name = "data/Counties_food_security.shp"

In [5]:
fs_df = gpd.read_file(file_name)

In [6]:
fs_df.head()

,COUNTY,Code,Severe_stu,Severe_was,Severe_und,Food_pover,Poor_Feedi,Borderline,Unacceptab,Maize_Prod,...,Food_Sou_1,Food_Sou_2,Food_sou_3,Access_imp,Access_to_,Access_to1,Average_Ra,Evapotrans,ObjectID,geometry
0,Turkana,23.0,7.8,4.7,15.9,52.7,19.0,24.0,43.0,3741.0,...,13.8,7.0,16.0,31.9,33.9,74.9,359.476373,0.194082,43,"POLYGON ((35.79593 5.34449, 35.79659 5.34468, ..."
1,Marsabit,10.0,16.1,11.5,11.5,23.8,2.0,3.0,15.0,347.0,...,11.5,13.4,7.5,26.0,42.0,68.5,382.080441,0.133693,25,"POLYGON ((36.05061 4.45622, 36.23184 4.45124, ..."
2,Mandera,9.0,18.5,5.8,6.2,38.9,3.0,11.0,12.0,0.0,...,5.2,10.6,9.0,40.9,37.2,72.6,314.380546,0.137762,24,"POLYGON ((41.62133 3.97673, 41.62272 3.97860, ..."
3,Wajir,8.0,8.3,1.8,2.6,10.5,3.0,10.0,6.0,1034.0,...,18.8,10.1,2.3,6.2,83.6,59.6,337.952920,0.148558,46,"POLYGON ((39.31812 3.47197, 39.31956 3.47168, ..."
4,West Pokot,24.0,14.8,7.8,7.1,26.2,3.0,22.0,25.0,61840.0,...,14.8,34.1,6.7,27.3,97.1,8.0,791.476830,0.436446,47,"POLYGON ((35.12745 2.62271, 35.12762 2.62302, ..."


In [7]:
fs_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   COUNTY      47 non-null     object  
 1   Code        47 non-null     float64 
 2   Severe_stu  47 non-null     float64 
 3   Severe_was  47 non-null     float64 
 4   Severe_und  47 non-null     float64 
 5   Food_pover  47 non-null     float64 
 6   Poor_Feedi  47 non-null     float64 
 7   Borderline  47 non-null     float64 
 8   Unacceptab  47 non-null     float64 
 9   Maize_Prod  47 non-null     float64 
 10  Beans_Prod  47 non-null     float64 
 11  TLU         47 non-null     float64 
 12  Land_Area_  47 non-null     float64 
 13  Arable_Lan  47 non-null     float64 
 14  Arable_L_1  47 non-null     float64 
 15  Rural_Popu  47 non-null     float64 
 16  Urban_Popu  47 non-null     float64 
 17  Total_Popu  47 non-null     float64 
 18  Population  47 non-null     float64 
 19  Ma

In [8]:
fs_df.describe()

,Code,Severe_stu,Severe_was,Severe_und,Food_pover,Poor_Feedi,Borderline,Unacceptab,Maize_Prod,Beans_Prod,...,Food_Sourc,Food_Sou_1,Food_Sou_2,Food_sou_3,Access_imp,Access_to_,Access_to1,Average_Ra,Evapotrans,ObjectID
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,...,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,24.000000,11.308511,3.070213,3.117021,10.644681,2.212766,10.787234,13.170213,84324.531915,16297.361702,...,64.134043,9.304255,20.653191,5.859574,59.119149,65.329787,40.874468,1026.693364,0.597710,24.000000
std,13.711309,3.642015,2.909872,3.983680,11.091160,2.873690,5.225091,7.111898,109516.388380,18278.600955,...,11.254956,6.632888,9.536015,3.082677,24.412039,17.932578,24.389105,417.020621,0.261364,13.711309
min,1.000000,3.000000,0.000000,0.000000,0.200000,0.000000,2.000000,3.000000,0.000000,0.000000,...,38.000000,2.200000,0.400000,1.900000,6.200000,27.800000,1.300000,314.380546,0.133693,1.000000
25%,12.500000,8.400000,1.050000,0.750000,3.850000,1.000000,7.500000,9.000000,19746.500000,251.000000,...,57.150000,5.950000,13.300000,4.000000,39.850000,53.150000,17.550000,732.168188,0.422925,12.500000
50%,24.000000,11.100000,2.100000,1.600000,7.000000,2.000000,10.000000,12.000000,54675.000000,10827.000000,...,61.900000,8.200000,21.500000,5.100000,60.600000,67.200000,39.300000,1011.279505,0.631576,24.000000
75%,35.500000,13.600000,4.500000,2.900000,11.750000,3.000000,13.000000,15.500000,91821.000000,26643.500000,...,70.450000,10.400000,28.550000,6.800000,82.500000,79.850000,60.150000,1339.134350,0.822180,35.500000
max,47.000000,18.500000,11.900000,16.600000,52.700000,19.000000,24.000000,43.000000,548197.000000,83945.000000,...,88.900000,47.200000,38.100000,17.000000,98.800000,97.100000,93.700000,1857.249231,1.024794,47.000000
